# Prueba técnica Servinformación

El objetivo de este proyecto es mostrar y aplicar algunas habilidades en el área de ciencia de datos, con enfoque en la aplicación de modelos de Machine Learning.

In [1]:
# Importamos librerías
import kaggle
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, accuracy_score
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import joblib
import utils as ut
import nbformat
import warnings

warnings.filterwarnings("ignore")

### Cargamos los datasets a usar

In [2]:
# Cargamos los datasets

df_categorical = pd.read_csv("data/df_categorical.csv")
df_filtered = pd.read_csv("data/df_filtered.csv")
df_numerical = pd.read_csv("data/df_numerical.csv")
df_non_categorical = pd.read_csv("data/df_non_categorical.csv")

# Modelos ML

El modelos propuesto es CatBoost, este modelo nos permite trabajar con datos categóricos y también tiene la capacidad de manejar datos desbalanceados sin tener que realizar reescalamiento con mayor eficiencia que un árbol de decisión. 

In [3]:
df_filtered.drop("name", inplace=True, axis=1)  # Eliminamos la columna name

Ahora almaceno las variables del modelo (X, y)

In [4]:
X = df_filtered.drop(
    ["log_price", "amenities"], axis=1
)  # Eliminamos la variable objetivo y las amenities
y = df_filtered["log_price"]  # Definimos la variable objetivo que el precio

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Se almacenan las variables categóricas y numéricas para más adelante crear un pipeline de datos, que me simplifica la estandarización y creación de dummies para las variables

In [5]:
CATERGORICAL_COLUMNS = list(df_categorical.columns)
CATERGORICAL_COLUMNS.remove("amenities")
CATERGORICAL_COLUMNS.remove("name")
CATERGORICAL_COLUMNS

['property_type',
 'room_type',
 'bed_type',
 'cancellation_policy',
 'city',
 'neighbourhood']

In [6]:
NUMERICAL_COLUMNS = list(df_numerical.columns)
NUMERICAL_COLUMNS.remove("log_price")
NUMERICAL_COLUMNS

['accommodates',
 'bathrooms',
 'latitude',
 'longitude',
 'number_of_reviews',
 'bedrooms',
 'beds']

Estructuro el Pipeline para la estandarización de las variables numéricas y la creación de dummies de las variables categóricas

In [8]:
numerical_pipeline = make_pipeline(StandardScaler())  # Normalizamos los datos numéricos

catergorical_pipeline = make_pipeline(
    OneHotEncoder()
)  # Codificamos las variables categóricas

full_pipeline = make_column_transformer(
    (numerical_pipeline, NUMERICAL_COLUMNS),
    (catergorical_pipeline, CATERGORICAL_COLUMNS),
)  # Unos los dos pipelines en uno solo para poder llevar a cabo el proceso de entrenamiento

Luego del pipeline, aplicamos los tres modelos, los cuales son:

* XgBoost
* RandomForest
* CatBoost

| Modelo       | Ventajas                                                                                                                                     | Desventajas                                                                                                                |
|--------------|----------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------|
| **XGBoost**  | - Alta precisión en predicciones.                                                                                                            | - Puede ser más lento de entrenar debido a su complejidad.                                                                 |
|              | - Maneja datos faltantes de manera eficiente.                                                                                                | - Requiere ajustes cuidadosos de hiperparámetros.                                                                          |
|              | - Implementación optimizada y paralelización.                                                                                                | - Sensible a los datos desbalanceados si no se manejan adecuadamente.                                                      |
|              | - Regularización incorporada para evitar el sobreajuste.                                                                                     |                                                                                                                            |
| **RandomForest** | - Robusto a overfitting debido a la combinación de múltiples árboles de decisión.                                                          | - Menor precisión comparado con métodos más avanzados como XGBoost y CatBoost.                                             |
|              | - Fácil de entender e implementar.                                                                                                           | - Puede ser menos eficiente en tiempo y memoria para datasets grandes.                                                     |
|              | - Capaz de manejar grandes cantidades de características y muestras.                                                                         | - No maneja características categóricas automáticamente.                                                                  |
|              | - Menos sensible a las configuraciones de hiperparámetros.                                                                                   |                                                                                                                            |
| **CatBoost** | - Maneja características categóricas sin necesidad de preprocesamiento.                                                                      | - Puede ser más complejo de entender y configurar comparado con RandomForest.                                              |
|              | - Menor necesidad de ajuste de hiperparámetros.                                                                                              | - A pesar de ser rápido en inferencia, el tiempo de entrenamiento puede ser considerablemente largo para datasets grandes. |
|              | - Regularización incorporada y manejo eficiente de datos faltantes.                                                                          | - Es una biblioteca relativamente nueva y puede tener menos recursos de aprendizaje disponibles en comparación con XGBoost.|
|              | - Buen rendimiento en datasets con características categóricas y numéricas mixtas.                                                           |                                                                                                                            |


In [10]:
models = {
    "XGBoost": make_pipeline(full_pipeline, XGBRegressor()),
    "RandomForest": make_pipeline(full_pipeline, RandomForestRegressor()),
    "CatBoost": make_pipeline(
        full_pipeline, CatBoostRegressor(verbose=False), memory="airbnb_model_cache"
    ),
}

model_list = []
model_name_list = []
rmse_list = []


for model_name, model in models.items():
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    r2 = r2_score(y_pred, y_test)
    model_list.append(model)
    model_name_list.append(model_name)
    rmse_list.append(r2)


_ = {"Model Name": model_name_list, "R² Score": rmse_list}

results = pd.DataFrame.from_dict(_).sort_values(by="R² Score", ascending=False)

results

,Model Name,R² Score
1,RandomForest,0.574898
2,CatBoost,0.573185
0,XGBoost,0.562790


Después de analizar los resultados de la aplicación de los tres modelos, se destaca que el desempeño del modelo RandomForest fue ligeramente superior al de los otros dos, seguido de cerca por CatBoost, con XGBoost mostrando un rendimiento algo más bajo.

Aunque los resultados no son extraordinarios, ya que los modelos solo lograron predecir aproximadamente el 60% de la variabilidad de los datos, existen varias estrategias que podrían mejorarlos. Es crucial revisar el preprocesamiento de los datos y asegurarse de utilizar la información correcta. Además, se recomienda realizar una validación cruzada para evaluar el rendimiento del modelo de manera más robusta, así como revisar detenidamente las características que fueron incluidas o excluidas en el proceso de modelado. Considerar la recopilación de más información también podría ser beneficioso para fortalecer el rendimiento predictivo.

En este contexto, se optará por ajustar los hiperparámetros del modelo CatBoost. Esta decisión se basa en su mejor rendimiento en comparación con RandomForest y en su capacidad para manejar automáticamente ciertos aspectos de la ingeniería de características, lo que podría estar afectando la precisión de las predicciones en el caso de XGBoost.

In [17]:
param_grid = {
    "catboostregressor__depth": np.arange(3, 10).tolist(),
    "catboostregressor__l2_leaf_reg": np.arange(2, 10).tolist(),
    "catboostregressor__random_strength": np.arange(0, 10, 2).tolist(),
}

fine_tuned_model = RandomizedSearchCV(
    model_list[2],
    param_grid,
    cv=5,
    n_jobs=-1,
    scoring="r2",
    n_iter=10,
)
fine_tuned_model.fit(X_train, y_train)

/mnt/ssd-storage/0-ivan-work/prueba-servinformacion/ve-servi/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/mnt/ssd-storage/0-ivan-work/prueba-servinformacion/ve-servi/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/mnt/ssd-storage/0-ivan-work/prueba-servinformacion/ve-servi/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/mnt/ssd-storage/0-ivan-work/prueba-servinformacion/ve-servi/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/mnt/ssd-storage/0-ivan-work/prueba-servinformacion/ve-servi/lib/python3.

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(memory='airbnb_model_cache',
                                      steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('pipeline-1',
                                                                               Pipeline(steps=[('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['accommodates',
                                                                                'bathrooms',
                                                                                'latitude',
                                                                                'longitude',
                                                                                'number_of_reviews',
                                                                                'bedrooms',
                                                                                'beds']),
                                                                              ('pipeline-2',
                                                                               Pipeline(steps=[('onehotencoder',
                                                                                                OneHotEncoder...
                                                                               ['property_type',
                                                                                'room_type',
                                                                                'bed_type',
                                                                                'cancellation_policy',
                                                                                'city',
                                                                                'neighbourhood'])])),
                                             ('catboostregressor',
                                              <catboost.core.CatBoostRegressor object at 0x7f6f0db03880>)]),
                   n_jobs=-1,
                   param_distributions={'catboostregressor__depth': [3, 4, 5, 6,
                                                                     7, 8, 9],
                                        'catboostregressor__l2_leaf_reg': [2, 3,
                                                                           4, 5,
                                                                           6, 7,
                                                                           8,
                                                                           9],
                                        'catboostregressor__random_strength': [0,
                                                                               2,
                                                                               4,
                                                                               6,
                                                                               8]},
                   scoring='r2')

In [13]:
fine_tuned_model.best_params_

{'catboostregressor__random_strength': 8,
 'catboostregressor__l2_leaf_reg': 3,
 'catboostregressor__depth': 5}

In [14]:
y_pred_fine_tuned = fine_tuned_model.predict(X_test)
# y_pred_fine_tuned = airbnb_model.predict(X_test)

In [15]:
r2_fine_tuned = r2_score(y_test, y_pred_fine_tuned) * 100

print(f"R² score:  {r2_fine_tuned}%")

R² score:  68.99587141298807%


In [16]:
scatter_trace = go.Scatter(
    x=y_test,
    y=y_pred_fine_tuned,
    mode="markers",
    name="Actual vs. Predicted",
    line=dict(color="#1f6d8f"),
    showlegend=False,
)

line_trace = go.Scatter(
    x=y_test,
    y=y_test,
    mode="lines",
    name="Regreesion Line",
    showlegend=False,
)

fig = go.Figure()

fig.update_layout(
    title="Tuned CatBoost Regressor (R2 score: {:.2f}%)".format(r2_fine_tuned),
    xaxis_title="Actual Log Price",
    yaxis_title="Predicted Log Price",
    height=900,
    width=900,
)

fig.add_trace(scatter_trace)
fig.add_trace(line_trace)

fig.show()

In [18]:
joblib.dump(fine_tuned_model, "data/airbnb_model.pkl")

['data/airbnb_model.pkl']

In [19]:
airbnb_model = joblib.load("data/airbnb_model.pkl")